In [1]:
import os
import sys
import time
import pickle
import random
import numpy
import tensorflow as tf
from tensorflow import keras,RaggedTensor
from tensorflow.keras.layers import Input,LSTM,Dense,MaxPool1D,Bidirectional
from tensorflow.keras.models import Model

In [2]:
def load_from_file(file_path):
	with open(file_path, "rb") as file:
		return pickle.load(file)

In [3]:
class EnhancedEmbedding(tf.keras.layers.Embedding):
    def __init__(self, input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None, **kwargs):
        super().__init__(input_dim, output_dim, embeddings_initializer, embeddings_regularizer, activity_regularizer, embeddings_constraint, mask_zero, input_length, **kwargs)
        self.five=tf.constant(0.5)
    
    # @tf.function
    def embedding(self,inputs):
        return super().call(inputs)
    
    # @tf.function
    def map_2(self,tokens):
        identifier=self.embedding(tokens[0])
        cur_word=self.embedding(tokens[2])
        return tf.squeeze(tf.reduce_mean(identifier)*self.five+cur_word*self.five)
    # @tf.function
    def map_1(self,inputs):
        return tf.map_fn(fn=lambda x :self.map_2(x),elems=inputs,dtype=tf.float32)

    def call(self, inputs):
        final_embeddings=tf.map_fn(fn=lambda x:self.map_1(x),elems=inputs,dtype=tf.float32)

        return final_embeddings

In [4]:
class EnhancedModel(Model):
    def __init__(self,  embedding_dim, hidden_dim, vocab_size, label_size,seq_len, pretrained_weight):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.label_size = label_size
        self.activation = tf.keras.activations.tanh
        self.num_layers = 1

        self.embedding=EnhancedEmbedding(vocab_size,embedding_dim,embeddings_initializer=keras.initializers.Constant(pretrained_weight))
        # self.encoder = Bidirectional(LSTM(hidden_dim, return_sequences=True))
        self.encoder = Bidirectional(LSTM(hidden_dim, return_sequences=True,input_shape=(seq_len,embedding_dim)))
        self.pool=MaxPool1D(hidden_dim*2)
        self.decoder = Dense(self.label_size)

    def call(self, inputs):
        embeddings = self.embedding(inputs)
        # print("finish embedding")
        # print(embeddings.shape)
        lstm_out = self.encoder(embeddings)
        # print("finish lstm")
        # print(lstm_out.shape)
        lstm_out = tf.transpose(lstm_out, perm=[0,2,1])
        # print(lstm_out.shape)
        pool_out=self.pool(lstm_out)
        # print("pool shape")
        # print(pool_out.shape)
        out = tf.squeeze(pool_out,[1])
        # print("finish pool")
        # print(out.shape)
        out = self.decoder(out)
        # print("finish linear")
        return out

In [5]:
strategy = tf.distribute.get_strategy()
fix_pattern = 'InsertMissedStmt'
print("Fix pattern: {}".format(fix_pattern))
root = "../data/{}/".format(fix_pattern)
pretrain_vectors = load_from_file(os.path.join(root, "vectors.pkl"))

AUTO = tf.data.experimental.AUTOTUNE
SEQ_LEN=2
HIDDEN_DIM = 50
EPOCHS = 30
BATCH_SIZE = 64
LABELS = 2
USE_GPU = True
MAX_TOKENS = pretrain_vectors.shape[0]
EMBEDDING_DIM = pretrain_vectors.shape[1]

Fix pattern: InsertMissedStmt


In [6]:
train_x = load_from_file(os.path.join(root, "train/x_w2v_embed_more.pkl"))
val_x = load_from_file(os.path.join(root, "val/x_w2v_embed_more.pkl"))
test_x = load_from_file(os.path.join(root, "test/x_w2v_embed_more.pkl"))
	
train_y = load_from_file(os.path.join(root, "train/y_.pkl"))
val_y = load_from_file(os.path.join(root, "val/y_.pkl"))
test_y = load_from_file(os.path.join(root, "test/y_.pkl"))

In [ ]:
train_x=train_x[:64]
val_x=val_x[:64]
test_x=test_x[:64]

train_y=train_y[:64]
val_y=val_y[:64]
test_y=test_y[:64]

In [7]:
train_x_enc=tf.ragged.constant(train_x,dtype=tf.int32)
val_x_enc=tf.ragged.constant(val_x,dtype=tf.int32)
test_x_enc=tf.ragged.constant(test_x,dtype=tf.int32)

In [8]:
def built_model(train_data,val_data,batch_size,embedding_dim, hidden_dim, max_tokens, labels,seq_len, pretrain_weight):
    model = EnhancedModel(embedding_dim, hidden_dim, max_tokens, labels,seq_len, pretrain_weight)
    model.compile(optimizer=tf.optimizers.Adam(),loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath='./model',
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
    model.fit(
        train_data,
        steps_per_epoch=batch_size,
        validation_data=val_data,
        epochs=EPOCHS,
        callbacks=[model_checkpoint_callback]
        )
    return model

In [9]:
with strategy.scope():
    tf.config.run_functions_eagerly(True)
    train_dataset = (
        tf.data.Dataset.from_tensor_slices((train_x_enc, train_y))
            .repeat()
            .shuffle(2048)
            .batch(len(train_y)//BATCH_SIZE)
            .prefetch(AUTO)
    )
    valid_dataset = (
        tf.data.Dataset.from_tensor_slices((val_x_enc, val_y))
            .batch(len(val_y)//BATCH_SIZE)
            .cache()
            .prefetch(AUTO)
    )
    model=built_model(train_dataset,valid_dataset,BATCH_SIZE,EMBEDDING_DIM, HIDDEN_DIM, MAX_TOKENS, LABELS,SEQ_LEN, pretrain_vectors)
    

Epoch 1/30


In [ ]:
test_dataset = (
        tf.data.Dataset.from_tensor_slices((test_x_enc, test_y))
            .batch(len(test_y)//BATCH_SIZE)
            .cache()
            .prefetch(AUTO)
    )

In [6]:
# model = EnhancedModel(EMBEDDING_DIM, HIDDEN_DIM, MAX_TOKENS, LABELS,2, pretrain_vectors)
# model.compile(optimizer=tf.optimizers.Adam(),loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])

In [7]:
# x=tf.ragged.constant([[[[1,2],[],[2]],[[1,2],[],[2]]],[[[1,2],[],[2]],[[1,2],[],[2]]],[[[1,2],[],[2]],[[1,2],[],[2]]]],dtype=tf.int32)
# y=tf.convert_to_tensor([1,0,1],dtype=tf.int32)

In [8]:
tf.config.run_functions_eagerly(True)
model.fit(x,y)

1/1 [==============================] - 0s 2ms/step - loss: 10.2833 - accuracy: 0.3333


In [83]:
model.summary()

Model: "enhanced_model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 enhanced_embedding_17 (Enha  multiple                 819616    
 ncedEmbedding)                                                  
                                                                 
 lstm_17 (LSTM)              multiple                  16600     
                                                                 
 max_pooling1d_17 (MaxPoolin  multiple                 0         
 g1D)                                                            
                                                                 
 dense_17 (Dense)            multiple                  6         
                                                                 
Total params: 836,222
Trainable params: 836,222
Non-trainable params: 0
_________________________________________________________________


In [23]:
tf.data.Dataset.from_tensor_slices((x, y))

<TensorSliceDataset element_spec=(RaggedTensorSpec(TensorShape([None, None, None]), tf.int32, 2, tf.int64), TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [2]:
tf.config.list_physical_devices('GPU')

[]